# Volume 3: MongoDB.
    <Erin Matheson>
    <Vol 3, 403>
    <16 November, 2017>

In [89]:
import re
import json
from pymongo import MongoClient


In [98]:
#$ sudo mkdir -p /data/db   # Make a place for the server to store information
#$ sudo mongod              # Start the server up / run Mongo in the background

client = MongoClient()

In [99]:
db1 = client.db1
coll = db1.collection2

In [41]:
# client.database_names()

## Problem 1

The file `trump.json` contains posts from http://www.twitter.com (tweets) over the course of an hour that have the key word "trump".
Each line in the file is a single JSON message that can be loaded with `json.loads()`.

Create a MongoDB database and initialize a collection in the database.
Use the collection's `delete_many()` method with an empy set as input to clear existing contents of the collection, then fill the collection one line at a time with the data from `trump.json`.
Check that your collection has 95,643 entries with its `count()` method.

In [42]:
coll.delete_many({})

with open("trump.json", 'r') as infile:
    for line in infile:
        coll.insert_one(json.loads(line))

# so does it save this and I don't need to run it every time?

In [7]:
coll.count()
# count is 95643 

95643

## Problem 2

Query the Twitter collection for the following information.
- How many tweets include the word Russia? Use `re.IGNORECASE`.
- How many tweets came from one of the main continental US time zones? These are listed as `"Central Time (US & Canada)"`, `"Pacific Time (US & Canada)"`, `"Eastern Time (US & Canada)"`, and `"Mountain Time (US & Canada)"`.
- How often does each language occur? Construct a dictionary with each language and it’s frequency count.

In [79]:
#!head trump.json

In [82]:
coll.find_one({'coordinates': {'$ne': None}})
# not equals None

{'_id': ObjectId('5a1f158e13ff6f0e2513d715'),
 'contributors': None,
 'coordinates': {'coordinates': [19.4544773, 51.7765107], 'type': 'Point'},
 'created_at': 'Fri May 26 19:18:50 +0000 2017',
 'entities': {'hashtags': [{'indices': [0, 4], 'text': 'rus'}],
  'symbols': [],
  'urls': [{'display_url': 'pravda.ru/news/world/26-…',
    'expanded_url': 'https://www.pravda.ru/news/world/26-05-2017/1335543-trump-0/',
    'indices': [63, 86],
    'url': 'https://t.co/uXXD83llym'}],
  'user_mentions': []},
 'favorite_count': 0,
 'favorited': False,
 'filter_level': 'low',
 'geo': {'coordinates': [51.7765107, 19.4544773], 'type': 'Point'},
 'id': 868184671607296001,
 'id_str': '868184671607296001',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'is_quote_status': False,
 'lang': 'ru',
 'place': {'attributes': {},
  'bounding_box': {'coordinates': [[[19.320808, 51.686377],
    

In [43]:
results = coll.find({'text': re.compile('russia', re.IGNORECASE)})
results.count()
#5841

5841

In [100]:
Zone = coll.distinct('user.time_zone')
#print(Zone)
Zone = coll.find({'user.time_zone': {'$in' : ["Central Time (US & Canada)", "Pacific Time (US & Canada)", "Eastern Time (US & Canada)", "Mountain Time (US & Canada)" ]}})

print("{} tweets came from the 4 Main Time Zones".format(Zone.count()))

33939 tweets came from the 4 Main Time Zones


In [25]:
# Languages: lang

Lang = coll.find({"lang": {'$exists': True}})
Lang.count()

languages = coll.distinct("lang")
#print(languages)

lang_dict = {}

for l in languages:
    
    amount = coll.find({"lang": l})
    
    lang_dict[l] = amount.count()
      
print(lang_dict)

{'en': 84919, 'fr': 1235, 'es': 3027, 'tr': 588, 'nl': 212, 'pt': 632, 'und': 1653, 'it': 423, 'de': 635, 'da': 35, 'zh': 1, 'ru': 29, 'pl': 108, 'no': 21, 'eu': 6, 'tl': 48, 'in': 121, 'cs': 13, 'ht': 21, 'ro': 21, 'sl': 3, 'sv': 55, 'fi': 20, 'ar': 30, 'et': 23, 'lt': 8, 'hu': 6, 'el': 10, 'ja': 17, 'lv': 2, 'vi': 7, 'fa': 5, 'ko': 2, 'ur': 1, 'cy': 2, 'th': 3, 'is': 1, 'iw': 1, 'uk': 3}


## Problem 3

Query the Twitter collection from Problem 1 for the following information.
- What are the usernames of the 5 most popular (defined as having the most followers) tweeters? Don’t include repeats.
- Of the tweets containing at least 5 hashtags, sort the tweets by how early the 5th hashtag appears in the text. What is the earliest spot (character count) it appears?
- What are the coordinates of the tweet that came from the northernmost location? Use the latitude and longitude point in `"coordinates"`.

In [72]:
coll.find_one({})

{'_id': ObjectId('5a1f158c13ff6f0e2513d431'),
 'contributors': None,
 'coordinates': None,
 'created_at': 'Fri May 26 19:18:30 +0000 2017',
 'entities': {'hashtags': [{'indices': [112, 117], 'text': 'maga'},
   {'indices': [118, 129], 'text': 'trumptrain'},
   {'indices': [130, 136], 'text': 'trump'}],
  'symbols': [],
  'urls': [{'display_url': 'truthfeed.com/liberal-spin-d…',
    'expanded_url': 'http://truthfeed.com/liberal-spin-democrats-are-saying-that-losing-in-montana-is-a-win/76338/',
    'indices': [88, 111],
    'url': 'https://t.co/t6QJFs0i4S'}],
  'user_mentions': [{'id': 707278892801765377,
    'id_str': '707278892801765377',
    'indices': [3, 17],
    'name': 'TruthFeed News',
    'screen_name': 'TruthFeedNews'}]},
 'favorite_count': 0,
 'favorited': False,
 'filter_level': 'low',
 'geo': None,
 'id': 868184586844549121,
 'id_str': '868184586844549121',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_use

In [124]:
#users = coll.distinct("name")

# sort tweets by popularity:

TopTweets = coll.find({'name': {'$exists':True}})#.sort('followers_count', -1)
# .sort by () right after this ?
# if you do name can you only sort by name ???

Top_followed = coll.find({'user.followers_count': {'$exists': True}}).sort('user.followers_count', -1)#[:20]

Top_users = []
i = 0
while len(Top_users) != 5:
    
    tweet = Top_followed[i]

    if tweet['user']['name'] not in Top_users:
        Top_users.append(tweet['user']['name'])
        
    i+=1
    
print(list(Top_users))

['CNN', 'Reuters Top News', 'Fox News', 'Wall Street Journal', 'TIME']


In [139]:

''' Example
entities': {'hashtags': [{'indices': [112, 117], 'text': 'maga'},
   {'indices': [118, 129], 'text': 'trumptrain'},
   {'indices': [130, 136], 'text': 'trump'}],'''

hashtags = coll.find({'entities.hashtags.4': {'$exists' : True}})
                        # 4 is the 4th index/5th entry of a list

sorted_h = hashtags.sort('entities.hashtags.4.indices.0')
# sorted the cursor's results
#print((sorted_h.count()))
# can do .count on cursors

earliest = sorted_h[0]['entities']['hashtags'][4]['indices'][0]

print("The earliest 5th hashtag appears at the {}rd index.".format(earliest))
# sort the tweets by how early the FIFTH text appears

The earliest 5th hashtag appears at the 23rd index.


In [144]:

# Coordinates of northernmost tweet...

Coordinates = coll.find({'coordinates.coordinates': {'$ne': None}})
                                               # not equal to None
    
#print(Coordinates.count())

sort_coor = Coordinates.sort('coordinates.coordinates.0', -1)
Northern_most = sort_coor[0]['coordinates']['coordinates']

print(Northern_most)

# Latitude is horizontal and Should be 1st entry
#but this data set is unclear


20
[28.99237342, 41.06803023]


## Problem 4

Clean the Twitter collection in the following ways.

- Get rid of the `"retweeted_status"` field in each tweet.
- Update every tweet from someone with at least 1000 followers to include a popular field whose value is True. Report the number of popular tweets.



- (OPTIONAL) The geographical coordinates used before in coordinates.coordinates are turned off for most tweets. But many more have a bounding box around the coordinates in the place field. Update every tweet without coordinates that contains a bounding box so that the coordinates contains the average value of the points that form the bounding box. Make the structure of coordinates the same as the others, so it contains coordinates with a longitude, latitude array and a type, the value of which should be ’Point’.

In [105]:
# do i have to set it equal to something? No -it doesn't work
coll.update_many({}, {'$unset': {'retweeted_status': None}})
# changes it to None- gets rid of it


In [145]:
#print(coll.find_one().keys())

dict_keys(['_id', 'created_at', 'id', 'id_str', 'text', 'source', 'truncated', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'retweet_count', 'favorite_count', 'entities', 'favorited', 'retweeted', 'possibly_sensitive', 'filter_level', 'lang', 'timestamp_ms', 'popular'])


In [142]:

coll.update_many( {'user.followers_count': {'$gte': 1000}}, {'$set': {'popular': True}} )

Pop = coll.find({'popular': {'$exists': True}})
print("{} top popular tweets (tweets from users with 1000+ followers).".format(Pop.count()))


32489 top popular tweets (tweets from users with 1000+ followers).
